In [ ]:
from pathlib import Path
import xml.etree.ElementTree as ET

root = Path(".")
images_dir = root / "images"
labels_dir = root / "labels"
xml_path = root / "annotations.xml"

labels_dir.mkdir(parents=True, exist_ok=True)

tree = ET.parse(xml_path)
ann = tree.getroot()

written = 0
for img_node in ann.findall("image"):  # direct children in your XML
    img_rel = img_node.get("name")              # "images/0.png"
    img_name = Path(img_rel).name               # "0.png"
    stem = Path(img_name).stem                  # "0"

    w = float(img_node.get("width"))
    h = float(img_node.get("height"))

    yolo_lines = []
    for box in img_node.findall("box"):
        if box.get("label") != "strawberry":
            continue

        xmin = float(box.get("xtl"))
        ymin = float(box.get("ytl"))
        xmax = float(box.get("xbr"))
        ymax = float(box.get("ybr"))

        xc = ((xmin + xmax) / 2.0) / w
        yc = ((ymin + ymax) / 2.0) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        yolo_lines.append(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    (labels_dir / f"{stem}.txt").write_text("\n".join(yolo_lines) + ("\n" if yolo_lines else ""))
    written += 1

print(f"Wrote {written} label files into {labels_dir}/")


In [ ]:
!pip install ultralytics

In [ ]:
!yolo detect train data=strawberry.yaml model=yolov8n.pt imgsz=640 epochs=60 batch=8


In [ ]:
!yolo detect predict model=/home/rob/Programming/Datascience/runs/detect/train8/weights/best.pt source=test conf=0.3 iou=0.35 max_det=50 save=True


In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

pred_dir = Path("/home/rob/Programming/Datascience/runs/detect/predict25")

# change to pred_dir.rglob(...) if you want to include subfolders too
img_paths = sorted([p for p in pred_dir.iterdir()
                    if p.suffix.lower() in {".jpg", ".jpeg", ".png", ".bmp", ".webp"}])

print("Found images:", len(img_paths))

for p in img_paths:
    img = Image.open(p)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title(p.name)
    plt.show()


In [ ]:


import numpy as np

def extract_boxes(result):
    boxes = result.boxes.xyxy.cpu().numpy()   # [N, 4]
    scores = result.boxes.conf.cpu().numpy()  # [N]
    return boxes, scores

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    areaB = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])

    return inter / (areaA + areaB - inter + 1e-6)

def merge_contained_boxes(boxes, scores, cover_thr=0.6):
    used = set()
    merged = []

    idxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    for i in idxs:
        if i in used:
            continue

        group = [i]
        for j in idxs:
            if j != i and j not in used:
                if coverage(boxes[j], boxes[i]) >= cover_thr:
                    group.append(j)

        for g in group:
            used.add(g)

        merged_box = np.mean([boxes[g] for g in group], axis=0)
        merged_score = np.mean([scores[g] for g in group])

        merged.append((merged_box, merged_score))

    return merged


import cv2
import matplotlib.pyplot as plt

def draw(img, merged):
    for box, score in merged:
        x1,y1,x2,y2 = map(int, box)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(img, f"{score:.2f}", (x1,y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    return img


def filter_by_conf(merged, conf_thr=0.3):
    return [(box, score) for box, score in merged if score >= conf_thr]


def coverage(inner, outer):
    """
    How much of `inner` box is covered by `outer` box.
    Returns value in [0,1].
    """
    xA = max(inner[0], outer[0])
    yA = max(inner[1], outer[1])
    xB = min(inner[2], outer[2])
    yB = min(inner[3], outer[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    inner_area = (inner[2] - inner[0]) * (inner[3] - inner[1])

    return inter / (inner_area + 1e-6)

def suppress_contained_boxes(boxes, scores, cover_thr=0.6):
    """
    Removes boxes that are mostly contained inside stronger boxes.
    """
    idxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    keep = []

    for i in idxs:
        discard = False
        for k in keep:
            if coverage(boxes[i], boxes[k]) >= cover_thr:
                discard = True
                break
        if not discard:
            keep.append(i)

    return [(boxes[i], scores[i]) for i in keep]


In [ ]:
from ultralytics import YOLO

#model = YOLO("/home/rob/Programming/Datascience/runs/detect/train8/weights/best.pt")
model = YOLO("MODEL.pt")

results = model.predict(
    source="test",
    conf=0.25,        # low conf, let *you* filter
    iou=0.3,          # loose NMS, keep duplicates
    save=False
)

import cv2

for r in results:
    img = cv2.imread(r.path)

    boxes, scores = extract_boxes(r)

    merged = merge_contained_boxes(boxes, scores, cover_thr=0.6)

    merged = [(b,s) for b,s in merged if s >= 0.3]

    img = draw(img, merged)

    plt.figure(figsize=(10,6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"{len(merged)} strawberries detected")
    plt.show()

"""

image 1/5 /home/rob/Programming/Datascience/ING/PVI/cv13/test/DTB_Strawberry_Collection.jpg: 640x640 13 strawberrys, 248.6ms
image 2/5 /home/rob/Programming/Datascience/ING/PVI/cv13/test/are-organic-strawberry-plants-better-1.jpg: 448x640 5 strawberrys, 241.2ms
image 3/5 /home/rob/Programming/Datascience/ING/PVI/cv13/test/homegrown-strawberries-1080x675.jpg: 416x640 2 strawberrys, 125.9ms
image 4/5 /home/rob/Programming/Datascience/ING/PVI/cv13/test/nj.jpg: 352x640 9 strawberrys, 113.5ms
image 5/5 /home/rob/Programming/Datascience/ING/PVI/cv13/test/strawberry-bg-1300_800x.jpg: 640x640 12 strawberrys, 172.5ms
Speed: 14.8ms preprocess, 180.3ms inference, 5.7ms postprocess per image at shape (1, 3, 640, 640)
"""